In [2]:
# %load poi_id.py
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

/Users/Haleh/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Task 1: Select what features you'll use.
features_list is a list of strings, each of which is a feature name.<br>
The first feature must be "poi".<br>
You will need to use more features

In [3]:
### Task 1: Select what features you'll use.

features_list = ['poi',
                 'salary',
                 'deferral_payments',
                 'total_payments',
                 'loan_advances',
                 'bonus',
                 'restricted_stock_deferred',
                 'bonus_salary_R',
                 'restricted_stock_deferred',
                 'deferred_income',
                 'total_stock_value',
                 'expenses',
                 'exercised_stock_options',
                 'other',
                 'long_term_incentive',
                 'restricted_stock',
                 'director_fees',
                 'to_messages',
                 'from_poi_to_this_person',
                 'from_poi_to_this_person_Percent',                 
                 'from_messages',
                 'from_this_person_to_poi',
                 'from_this_person_to_poi_percent',
                 'shared_receipt_with_poi']


## Task 2: Remove outliers
based on mini-project in outlier lessons, i know there is a key "Total", which holds the total value for all other values. <br>
Another one is the key "THE TRAVEL AGENCY IN THE PARK".<br>
I'm going to exclude both of them



In [4]:
import pprint
names=[]
for k, v in data_dict.iteritems():
    names.append( k)
# pprint.pprint (names)
pprint.pprint (data_dict["TOTAL"], width=1)
# pprint.pprint (data_dict["THE TRAVEL AGENCY IN THE PARK"])

{'bonus': 97343619,
 'deferral_payments': 32083396,
 'deferred_income': -27992891,
 'director_fees': 1398517,
 'email_address': 'NaN',
 'exercised_stock_options': 311764000,
 'expenses': 5235198,
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 83925000,
 'long_term_incentive': 48521928,
 'other': 42667589,
 'poi': False,
 'restricted_stock': 130322299,
 'restricted_stock_deferred': -7576788,
 'salary': 26704229,
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 309886585,
 'total_stock_value': 434509511}


In [5]:
data_dict.pop("TOTAL")
data_dict.pop("THE TRAVEL AGENCY IN THE PARK")

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 362096,
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 362096,
 'total_stock_value': 'NaN'}

# Task 3: Create new feature(s)
Store to my_dataset for easy export below.

In [6]:
for key, value in data_dict.iteritems():
    if value['bonus'] == "NaN" or value['salary'] == "Nan":
        value['bonus_salary_R'] = "NaN"
    else:
        value['bonus_salary_R'] = float(value['bonus']) / float(value['salary'])

for key, value in data_dict.iteritems():
    if value["from_this_person_to_poi"] == "NaN" or value["to_messages"] == "Nan":
        value['from_this_person_to_poi_percent'] = "NaN"
    else:
        value['from_this_person_to_poi_percent'] = float(value['from_this_person_to_poi'])*100 / float(value['to_messages'])

for key, value in data_dict.iteritems():
    if value["from_poi_to_this_person"] == "NaN" or value["to_messages"] == "Nan":
        value['from_poi_to_this_person_Percent'] = "NaN"
    else:
        value['from_poi_to_this_person_Percent'] = float(value['from_poi_to_this_person'])*100 / float(value['to_messages'])
        


In [9]:
my_dataset = data_dict
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
        
print data_dict ['METTS MARK']


{'to_messages': 807, 'deferral_payments': 'NaN', 'expenses': 94299, 'poi': False, 'deferred_income': 'NaN', 'from_poi_to_this_person_Percent': 4.708798017348204, 'email_address': 'mark.metts@enron.com', 'long_term_incentive': 'NaN', 'restricted_stock_deferred': 'NaN', 'shared_receipt_with_poi': 702, 'loan_advances': 'NaN', 'from_messages': 29, 'other': 1740, 'director_fees': 'NaN', 'bonus': 600000, 'total_stock_value': 585062, 'from_poi_to_this_person': 38, 'bonus_salary_R': 1.6402943781644013, 'from_this_person_to_poi': 1, 'from_this_person_to_poi_percent': 0.12391573729863693, 'restricted_stock': 585062, 'salary': 365788, 'total_payments': 1061827, 'exercised_stock_options': 'NaN'}


# Task 4: Try a varity of classifiers
Please name your classifier clf for easy export below.
Note that if you want to do PCA or other multi-stage operations,
you'll need to use Pipelines. For more info:
http://scikit-learn.org/stable/modules/pipeline.html

In [7]:


# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)